In [1]:
import QuantLib as ql

Testing conversion from YoY cap-floor surface to YoY inflation term structure...

In [2]:
evaluationDate = ql.Date(23, 11, 2007)
ql.Settings.instance().evaluationDate = evaluationDate

In [3]:
nominalEUR = ql.YieldTermStructureHandle()
nominalGBP = ql.YieldTermStructureHandle()

yoyEU = ql.RelinkableYoYInflationTermStructureHandle()
yoyUK = ql.RelinkableYoYInflationTermStructureHandle()

cStrikesEU = []
fStrikesEU = []
cfMaturitiesEU = []
cPriceEU = ql.Matrix()
fPriceEU = ql.Matrix()

cStrikesUK = []
fStrikesUK = []
cfMaturitiesUK = []
cPriceUK = ql.Matrix()
fPriceUK = ql.Matrix()

In [4]:
yoyIndexUK = ql.YYUKRPIr(True, yoyUK)
yoyIndexEU = ql.YYEUHICPr(True, yoyEU)

In [5]:
timesEUR = [0.0109589, 0.0684932, 0.263014, 0.317808, 0.567123, 0.816438,
               1.06575, 1.31507, 1.56438, 2.0137, 3.01918, 4.01644,
               5.01644, 6.01644, 7.01644, 8.01644, 9.02192, 10.0192,
               12.0192, 15.0247, 20.0301, 25.0356, 30.0329, 40.0384,
               50.0466]
ratesEUR = [0.0415600, 0.0426840, 0.0470980, 0.0458506, 0.0449550, 0.0439784,
  0.0431887, 0.0426604, 0.0422925, 0.0424591, 0.0421477, 0.0421853,
  0.0424016, 0.0426969, 0.0430804, 0.0435011, 0.0439368, 0.0443825,
  0.0452589, 0.0463389, 0.0472636, 0.0473401, 0.0470629, 0.0461092,
  0.0450794]

timesGBP = [0.008219178, 0.010958904, 0.01369863,  0.019178082,  0.073972603,
  0.323287671, 0.57260274,  0.821917808, 1.071232877,  1.320547945,
  1.506849315, 2.002739726, 3.002739726, 4.002739726,  5.005479452,
  6.010958904, 7.008219178, 8.005479452, 9.008219178, 10.00821918,
  12.01369863, 15.0109589,  20.01369863, 25.01917808,  30.02191781,
  40.03287671, 50.03561644, 60.04109589, 70.04931507]
ratesGBP = [0.0577363, 0.0582314, 0.0585265, 0.0587165, 0.0596598,
       0.0612506, 0.0589676, 0.0570512, 0.0556147, 0.0546082,
       0.0549492, 0.053801, 0.0529333, 0.0524068, 0.0519712,
       0.0516615, 0.0513711, 0.0510433, 0.0507974, 0.0504833,
       0.0498998, 0.0490464, 0.04768, 0.0464862, 0.045452,
       0.0437699, 0.0425311, 0.0420073, 0.041151]


In [6]:
r = []
d = []

for time, rate in zip(timesEUR,ratesEUR):
    r+=[rate]
    ys=int(time)
    ds=int((time-ys)*365)
    dd=evaluationDate+ql.Period(ys,ql.Years)+ql.Period(ds,ql.Days)
    d+=[dd]

In [7]:
euriborTS = ql.CubicZeroCurve(d, r, ql.Actual365Fixed())

In [8]:
nominalHeur = ql.YieldTermStructureHandle(euriborTS)

In [9]:
nominalEUR = nominalHeur

In [10]:
d = []
r = []

for time, rate in zip(timesGBP,ratesGBP):
    r+=[rate]
    ys=int(time)
    ds=int((time-ys)*365)
    dd=evaluationDate+ql.Period(ys,ql.Years)+ql.Period(ds,ql.Days)
    d+=[dd]

In [11]:
gbpLiborTS = ql.CubicZeroCurve(d, r, ql.Actual365Fixed())
nominalHgbp = ql.YieldTermStructureHandle(gbpLiborTS)
nominalGBP = nominalHgbp

In [12]:
# times = years - lag, where the lag is 2 months or 2/12
# because this data is derived from cap/floor data that
# is based on a 2 month lag.

# note that these are NOT swap rates
# also not that the first value MUST be in the base period
# i.e. the first rate is for a negative time

In [13]:
yoyEUrates = [0.0237951,
     0.0238749, 0.0240334, 0.0241934, 0.0243567, 0.0245323,
     0.0247213, 0.0249348, 0.0251768, 0.0254337, 0.0257258,
     0.0260217, 0.0263006, 0.0265538, 0.0267803, 0.0269378,
     0.0270608, 0.0271363, 0.0272, 0.0272512, 0.0272927,
     0.027317, 0.0273615, 0.0273811, 0.0274063, 0.0274307,
     0.0274625, 0.027527, 0.0275952, 0.0276734, 0.027794]

In [14]:
d = []
r = []

baseDate = ql.TARGET().advance(evaluationDate, -2, ql.Months, ql.ModifiedFollowing)

for i, rate in enumerate(yoyEUrates):
    r+=[rate]
    dd= ql.TARGET().advance(baseDate, i, ql.Years, ql.ModifiedFollowing)
    d+=[dd]

In [15]:
indexIsInterpolated = True # actually false for UKRPI but smooth surfaces are
                           # better for finding intersections etc

In [16]:
pYTSEU = ql.YoYInflationCurve(evaluationDate, ql.TARGET(), ql.Actual365Fixed(), ql.Period(2, ql.Months), ql.Monthly, indexIsInterpolated, d, r)

In [17]:
yoyEU.linkTo(pYTSEU)

In [18]:
ncStrikesEU = 6
nfStrikesEU = 6
ncfMaturitiesEU = 7
capStrikesEU = [0.02, 0.025, 0.03, 0.035, 0.04, 0.05]
capMaturitiesEU = [ql.Period(i, ql.Years) for i in [3,4,7,10,15,20,30]]
capPricesEU =[[116.225, 204.945, 296.285, 434.29, 654.47, 844.775, 1132.33],
        [34.305, 71.575, 114.1, 184.33, 307.595, 421.395, 602.35],
        [6.37, 19.085, 35.635, 66.42, 127.69, 189.685, 296.195],
        [1.325, 5.745, 12.585, 26.945, 58.95, 94.08, 158.985],
        [0.501, 2.37, 5.38, 13.065, 31.91, 53.95, 96.97],
        [0.501, 0.695, 1.47, 4.415, 12.86, 23.75, 46.7]]

In [19]:
floorStrikesEU = [-0.01, 0.00, 0.005, 0.01, 0.015, 0.02]
floorPricesEU = [[0.501, 0.851, 2.44, 6.645, 16.23, 26.85, 46.365],
                [0.501, 2.236, 5.555, 13.075, 28.46, 44.525, 73.08],
                [1.025, 3.935, 9.095, 19.64, 39.93, 60.375, 96.02],
                [2.465, 7.885, 16.155, 31.6, 59.34, 86.21, 132.045],
                [6.9, 17.92, 32.085, 56.08, 95.95, 132.85, 194.18],
                [23.52, 47.625, 74.085, 114.355, 175.72, 229.565, 316.285]]

In [20]:
cStrikesEU = capStrikesEU
fStrikesEU = floorStrikesEU
cfMaturitiesEU = capMaturitiesEU

In [21]:
tcPriceEU = ql.Matrix(capPricesEU)
tfPriceEU = ql.Matrix(floorPricesEU)
cPriceEU = tcPriceEU
fPriceEU = tfPriceEU

In [22]:
# setupPriceSurface

# construct:
#  calendar, business day convention, and day counter are
#  taken from the nominal base give the reference date for
#  the inflation options (generally 2 or 3 months before
#  nominal reference date)

In [23]:
fixingDays = 0
lag = 3 # must be 3 because we use an interpolated index (EU)
yyLag = ql.Period(lag,ql.Months)
baseRate = 1 # not really used
dc = ql.Actual365Fixed()
cal = ql.TARGET()
bdc = ql.ModifiedFollowing
pn = nominalEUR.currentLink()
n = ql.YieldTermStructureHandle(pn)

In [24]:
cfEUprices = ql.YoYInflationCapFloorTermPriceSurface(fixingDays,
                               yyLag, yoyIndexEU, baseRate,
                               n, dc,
                               cal,    bdc,
                               cStrikesEU, fStrikesEU, cfMaturitiesEU,
                               cPriceEU, fPriceEU)

priceSurfEU = cfEUprices

In [25]:
testVol = ql.ConstantYoYOptionletVolatility(0.01, 2, ql.TARGET(), ql.ModifiedFollowing, ql.Actual365Fixed(),
ql.Period(3, ql.Months), ql.Annual, True, 0.01, 0.10)

In [26]:
hVS = ql.YoYOptionletVolatilitySurfaceHandle()

In [27]:
yoyPricerUD = ql.YoYInflationUnitDisplacedBlackCapFloorEngine(yoyIndexEU, hVS, nominalEUR)

In [28]:
yoyOptionletStripper = ql.InterpolatedYoYInflationOptionletStripper()

In [29]:
settlementDays = 0
cal = ql.TARGET()
bdc = ql.ModifiedFollowing
dc = ql.Actual365Fixed()

In [30]:
capFloorPrices = priceSurfEU

In [31]:
lag = priceSurfEU.observationLag()

In [32]:
slope = -0.5 

In [33]:
#when you have bad data, i.e. very low/constant
#prices for short dated extreem strikes
#then you cannot assume constant caplet vol
#(else arbitrage)
# N.B. if this is too extreme then can't
# get a no-arbitrage solution anyway
# the way the slope is used means that the slope is
# proportional to the level so higher slopes at
# the edges when things are more volatile

In [34]:
yoySurf = ql.KInterpolatedYoYInflationOptionletVolatilitySurface(settlementDays,
                cal, bdc, dc, lag, capFloorPrices, yoyPricerUD, yoyOptionletStripper, slope)

In [35]:
volATyear1 = [
      0.0128, 0.0093, 0.0083, 0.0073, 0.0064,
      0.0058, 0.0042, 0.0046, 0.0053, 0.0064,
      0.0098
]

In [36]:
volATyear3 = [
          0.0079, 0.0058, 0.0051, 0.0045, 0.0039,
          0.0035, 0.0026, 0.0028, 0.0033, 0.0039,
          0.0060
]

In [37]:
d = yoySurf.baseDate() + ql.Period(1,ql.Years)

In [38]:
d

Date(23,8,2008)

In [39]:
someSlice = yoySurf.Dslice(d)

In [40]:
for vol1, vol2 in zip(someSlice[1], volATyear1):
    print(vol1, vol2, vol1-vol2)

0.012757838362256007 0.0128 -4.216163774399366e-05
0.00931907485688127 0.0093 1.9074856881269933e-05
0.00829322971737263 0.0083 -6.770282627370852e-06
0.007316820579013168 0.0073 1.6820579013167997e-05
0.006381609813674116 0.0064 -1.8390186325884482e-05
0.005709352868561817 0.0058 -9.064713143818251e-05
0.0041311939892404136 0.0042 -6.880601075958618e-05
0.004587204307701849 0.0046 -1.2795692298150571e-05
0.005322986425009469 0.0053 2.2986425009468604e-05
0.006391228051211841 0.0064 -8.771948788159449e-06
0.009758167514944165 0.0098 -4.183248505583437e-05


In [41]:
d = yoySurf.baseDate() + ql.Period(3,ql.Years)

In [42]:
someOtherSlice = yoySurf.Dslice(d)

In [43]:
for vol1, vol2 in zip(someOtherSlice[1], volATyear3):
    print(vol1, vol2, vol1-vol2)

0.007859967336391962 0.0079 -4.003266360803892e-05
0.005741679869739384 0.0058 -5.832013026061539e-05
0.005105858826016777 0.0051 5.858826016776758e-06
0.004500066403007662 0.0045 6.640300766191931e-08
0.003919394352907175 0.0039 1.939435290717554e-05
0.003499731682437454 0.0035 -2.6831756254602074e-07
0.0025279456176045306 0.0026 -7.205438239546924e-05
0.0028083852974516424 0.0028 8.385297451642387e-06
0.0032636616494645843 0.0033 -3.633835053541571e-05
0.003923961946674869 0.0039 2.3961946674869e-05
0.005997293558841497 0.006 -2.7064411585028467e-06


In [45]:
yyATMt  = priceSurfEU.atmYoYSwapTimeRates()

In [46]:
yyATMt

((3.0027397260273974,
  4.002739726027397,
  7.005479452054795,
  10.008219178082191,
  15.01095890410959,
  20.013698630136986,
  30.02191780821918),
 (0.024585990653559164,
  0.02475752056067443,
  0.024939600099273256,
  0.02525957610999954,
  0.02584977670587058,
  0.026288266145827632,
  0.026791525496631604))